In [11]:
import sys
import os
from cobra.io import read_sbml_model

sys.path.append(os.path.abspath("../../src"))

model = read_sbml_model("/Users/azddza/Electromics-project/models/gap_but_models/added_butyrate_exchange_reactions/MAG037_gapfilled_noO2.xml")

print(f"模型中包含 {len(model.reactions)} 个反应，{len(model.metabolites)} 个代谢物，{len(model.genes)} 个基因。")


模型中包含 702 个反应，688 个代谢物，344 个基因。


In [2]:
print(model.metabolites.get_by_id("cpd00211_e0").reactions)

frozenset({<Reaction TR_cpd00211_c0 at 0x17c4e68c0>, <Reaction EX_cpd00211_e0 at 0x17c4e6740>})


In [10]:
for rxn in model.reactions:
    if "cpd00211" in rxn.id:
        print(rxn.id, rxn.reaction)

EX_cpd00211_e0 cpd00211_e0 <=> 
TR_cpd00211_c0 cpd00211_e0 <=> cpd00211_c0


In [29]:

tr = model.reactions.get_by_id("TR_cpd00211_c0")
ex = model.reactions.get_by_id("EX_cpd00211_e0")

# 打印上下限信息
print(" TR_cpd00211_c0 (转运反应):")
print("  Reaction:", tr.reaction)
print("  Bounds:", tr.lower_bound, "→", tr.upper_bound)

print("\n EX_cpd00211_e0 (exchange反应):")
print("  Reaction:", ex.reaction)
print("  Bounds:", ex.lower_bound, "→", ex.upper_bound)


 TR_cpd00211_c0 (转运反应):
  Reaction: cpd00211_e0 <=> cpd00211_c0
  Bounds: -1000.0 → 1000.0

 EX_cpd00211_e0 (exchange反应):
  Reaction: cpd00211_e0 <=> 
  Bounds: -1000.0 → 1000.0


In [13]:
import networkx as nx
G_model = nx.DiGraph()
for reaction in model.reactions:
    reactants = [met.id for met in reaction.reactants]
    products = [met.id for met in reaction.products]
    for r in reactants:
        for p in products:
            G_model.add_edge(r, p, reaction=reaction.id)

In [5]:
print("前10个代谢物节点:")
for i, node in enumerate(G_model.nodes):
    print(node)
    if i > 10: break

print("\n前10条反应边（从代谢物A → B）:")
for i, (u, v, data) in enumerate(G_model.edges(data=True)):
    print(f"{u} → {v} via reaction: {data.get('reaction')}")
    if i > 10: break


前10个代谢物节点:
cpd00002_c0
cpd00012_c0
cpd00018_c0
cpd00067_c0
cpd00279_c0
cpd00010_c0
cpd00142_c0
cpd00031_c0
cpd00001_c0
cpd00295_c0
cpd11420_c0
cpd11421_c0

前10条反应边（从代谢物A → B）:
cpd00002_c0 → cpd00012_c0 via reaction: rxn30387_c0
cpd00002_c0 → cpd00018_c0 via reaction: rxn30387_c0
cpd00002_c0 → cpd00067_c0 via reaction: rxn10481_c0
cpd00002_c0 → cpd00279_c0 via reaction: rxn00988_c0
cpd00002_c0 → cpd14955_c0 via reaction: rxn07584_c0
cpd00002_c0 → cpd00327_c0 via reaction: rxn09449_c0
cpd00002_c0 → cpd00297_c0 via reaction: rxn01513_c0
cpd00002_c0 → cpd00008_c0 via reaction: rxn10481_c0
cpd00002_c0 → cpd00009_c0 via reaction: rxn10481_c0
cpd00002_c0 → cpd00053_c0 via reaction: rxn00187_c0
cpd00002_c0 → cpd15603_c0 via reaction: rxn12848_c0
cpd00002_c0 → cpd11437_c0 via reaction: rxn05249_c0


In [14]:
import networkx as nx

# 读取社区网络图
G = nx.read_graphml("cpd00211_community.graphml")  
print(f"图节点数: {G.number_of_nodes()}, 边数: {G.number_of_edges()}")


图节点数: 20151, 边数: 147336


In [15]:
# 从社区图中取出所有代谢物名称（需要确保命名方式一致，如带_c）
community_mets = set(G.nodes)

# 在模型图中找这些代谢物是否能从某些模型已有代谢物可达
exchange_mets = set()
for rxn in model.exchanges:
    for met in rxn.metabolites:
        exchange_mets.add(met.id)

# 在模型图中，从每个 exchange 代谢物出发，检查能否到目标社区
valid_starts = []
for start in exchange_mets:
    for target in community_mets:
        try:
            if nx.has_path(G_model, start, target):
                valid_starts.append(start)
                break
        except:
            continue


/Users/azddza/Library/Caches/pypoetry/virtualenvs/electromics-project-Ddk0_V8b-py3.10/lib/python3.10/site-packages/cobra/medium/boundary_types.py:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  compartment = most[matches[most]][0]
There are several compartments that look like an external compartment but `e0` has the most boundary reactions, so using that as the external compartment.


In [16]:
print("推荐用于 gapfilling 的起点代谢物（在模型中存在且连通社区）:")
for met in valid_starts:
    print(met)


推荐用于 gapfilling 的起点代谢物（在模型中存在且连通社区）:
cpd00001_e0
cpd00118_e0
cpd15603_e0
cpd00011_e0
cpd00067_e0
cpd00531_e0
cpd00508_e0
cpd00063_e0
cpd00007_e0
cpd00028_e0
cpd00129_e0
cpd00305_e0
cpd00264_e0
cpd04097_e0
cpd15240_e0
cpd00149_e0
cpd00058_e0
cpd00211_e0
cpd00254_e0
cpd00048_e0
cpd00119_e0
cpd03586_e0
cpd29426_e0
cpd00116_e0
cpd10515_e0
cpd00034_e0
cpd03453_e0
cpd00104_e0
cpd03422_e0
cpd00030_e0
cpd15560_e0
cpd00220_e0
cpd00971_e0
cpd00009_e0
cpd00092_e0
cpd03726_e0
cpd00156_e0
cpd00166_e0
cpd00205_e0
cpd00066_e0
cpd10516_e0
cpd01012_e0


In [17]:
import pickle
reaction_dict_path = "reaction_dict.pkl"

# ===== 加载 reaction_dict =====
with open(reaction_dict_path, "rb") as f:
    reaction_dict = pickle.load(f)


In [19]:
###最短路径 gapfilling

from cobra import Reaction, Metabolite
import networkx as nx
from cobra.util.solver import linear_reaction_coefficients
from cobra.flux_analysis import find_blocked_reactions

# 自定义 dead-end 检查函数
def find_dead_ends_manual(model):
    dead_ends = []
    for met in model.metabolites:
        producing = any(rxn.metabolites[met] > 0 for rxn in met.reactions)
        consuming = any(rxn.metabolites[met] < 0 for rxn in met.reactions)
        if not (producing and consuming):
            dead_ends.append(met.id)
    return dead_ends

target_met = "cpd00211_c0"
results = []

for start in valid_starts:  
    
    print(f"\n▶ 起点: {start}")
    try:
        path = nx.shortest_path(G, source=start, target=target_met)
        print(f" 路径: {path}")
    except Exception as e:
        print(f" 搜索异常: {e}")
        continue

    rxn_ids = []
    for i in range(len(path) - 1):
        try:
            rxn_id = G[path[i]][path[i + 1]]["reaction"]
            rxn_ids.append(rxn_id)
        except:
            print(f" 边 {path[i]} → {path[i+1]} 缺失 reaction")
            continue

    print(f" 路径反应 IDs: {rxn_ids}")

    existing_rxns = set(r.id for r in model.reactions)
    to_add_rxns = [rxn for rxn in rxn_ids if rxn not in existing_rxns]
    print(f" 缺失反应: {to_add_rxns}")

    with model.copy() as m:
        for rxn_id in to_add_rxns:
            if rxn_id in reaction_dict:
                new_rxn = reaction_dict[rxn_id].copy()
                m.add_reactions([new_rxn])
                print(f" 添加反应: {rxn_id}")
            else:
                print(f" reaction_dict 缺失反应: {rxn_id}")

        ex_id = f"EX_{start}"
        if ex_id in m.reactions:
            m.reactions.get_by_id(ex_id).lower_bound = -10
        else:
            try:
                ex = Reaction(ex_id)
                ex.name = f"Exchange for {start}"
                ex.lower_bound = -10
                ex.upper_bound = 1000
                ex.add_metabolites({m.metabolites.get_by_id(start): -1})
                m.add_reactions([ex])
                print(f" 添加 exchange: {ex_id}")
            except KeyError:
                print(f" 找不到代谢物: {start}")
                continue

        target_rxn_id = "EX_cpd00211_e0"
        if target_rxn_id in m.reactions:
            m.objective = m.reactions.get_by_id(target_rxn_id)
        else:
            try:
                target_met_obj = m.metabolites.get_by_id("cpd00211_e0")
                target_ex = Reaction(target_rxn_id)
                target_ex.name = f"Exchange for cpd00211_e0"
                target_ex.lower_bound = 0
                target_ex.upper_bound = 1000
                target_ex.add_metabolites({target_met_obj: -1})
                m.add_reactions([target_ex])
                m.objective = target_ex
                print(f"添加目标 exchange: {target_rxn_id}")
            except KeyError:
                print(f" 无法添加目标 exchange，缺少代谢物: cpd00211_e0")
                continue

        sol = m.optimize()
        flux = sol.objective_value if sol.status == "optimal" else 0
        print(f" 通量结果: {flux}")

        blocked_rxns = set(find_blocked_reactions(m))
        blocked_in_path = [r for r in rxn_ids if r in blocked_rxns]
        if blocked_in_path:
            print(f" 路径中有被阻断的反应: {blocked_in_path}")

        dead_mets = set(find_dead_ends_manual(m))
        mets_in_path = set()
        for rxn_id in rxn_ids:
            rxn = m.reactions.get_by_id(rxn_id)
            mets_in_path.update(m.id for m in rxn.metabolites)
        dead_in_path = [m for m in mets_in_path if m in dead_mets]
        if dead_in_path:
            print(f" 路径中存在 dead-end 代谢物: {dead_in_path}")

        print(" 各反应通量:")
        for rxn_id in rxn_ids:
            try:
                flux_val = sol.fluxes.get(rxn_id, 0)
                print(f"  - {rxn_id}: {flux_val}")
            except:
                print(f"  - {rxn_id}: 不在模型中")

        results.append({
            "start": start,
            "path": path,
            "missing_reactions": to_add_rxns,
            "num_missing": len(to_add_rxns),
            "flux": flux,
            "blocked_reactions": blocked_in_path,
            "dead_end_metabolites": dead_in_path,
        })



▶ 起点: cpd00001_e0
 路径: ['cpd00001_e0', 'cpd00001_c0', 'cpd00211_c0']
 路径反应 IDs: ['rxn34181', 'rxn45696']
 缺失反应: ['rxn34181', 'rxn45696']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpxzd1nsm6.lp
Reading time = 0.01 seconds
: 688 rows, 1404 columns, 6586 nonzeros
 添加反应: rxn34181
 添加反应: rxn45696
 通量结果: 0.0
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmph1lv_c3q.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6603 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpugv3ypk6.lp
Reading time = 0.00 seconds
: 691 rows, 1409 columns, 6603 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Ac

In [21]:
all_zero = all(entry['flux'] == 0 for entry in results)
print(f"是否所有 flux 都为 0: {all_zero}")

是否所有 flux 都为 0: True


In [22]:
len(results)

42

In [24]:
def score_path(item, alpha=1.0, beta=2.0, gamma=1.5, delta=0.5):
    """
    评分函数：得分越高越差
    alpha：缺失反应权重
    beta：阻塞反应权重
    gamma：死胡同权重
    delta：路径长度权重
    """
    score = (
        alpha * item['num_missing'] +
        beta * len(item.get('blocked_reactions', [])) +
        gamma * len(item.get('dead_end_metabolites', [])) +
        delta * len(item.get('path', []))
    )
    return score

ranked = sorted(results, key=lambda x: score_path(x))

# 打印
for i, item in enumerate(ranked[:3]):
    print(f"Rank {i+1}: Start = {item['start']}, Score = {score_path(item)}")
    print(f"  Missing: {item['missing_reactions']}")
    print(f"  Blocked: {item['blocked_reactions']}")
    print(f"  Dead-end: {item['dead_end_metabolites']}")
    print()


Rank 1: Start = cpd00067_e0, Score = 5.0
  Missing: ['rxn08183']
  Blocked: []
  Dead-end: ['cpd00211_c0', 'cpd00211_e0']

Rank 2: Start = cpd00211_e0, Score = 5.0
  Missing: ['rxn11382']
  Blocked: []
  Dead-end: ['cpd00211_e0', 'cpd00211_c0']

Rank 3: Start = cpd00028_e0, Score = 7.0
  Missing: ['rxn05148', 'rxn01236']
  Blocked: ['rxn01236']
  Dead-end: ['cpd01662_c0']



In [26]:
ranked

[{'start': 'cpd00067_e0',
  'path': ['cpd00067_e0', 'cpd00211_c0'],
  'missing_reactions': ['rxn08183'],
  'num_missing': 1,
  'flux': 0.0,
  'blocked_reactions': [],
  'dead_end_metabolites': ['cpd00211_c0', 'cpd00211_e0']},
 {'start': 'cpd00211_e0',
  'path': ['cpd00211_e0', 'cpd00211_c0'],
  'missing_reactions': ['rxn11382'],
  'num_missing': 1,
  'flux': 0.0,
  'blocked_reactions': [],
  'dead_end_metabolites': ['cpd00211_e0', 'cpd00211_c0']},
 {'start': 'cpd00028_e0',
  'path': ['cpd00028_e0', 'cpd00008_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn05148', 'rxn01236'],
  'num_missing': 2,
  'flux': 0.0,
  'blocked_reactions': ['rxn01236'],
  'dead_end_metabolites': ['cpd01662_c0']},
 {'start': 'cpd00305_e0',
  'path': ['cpd00305_e0', 'cpd00008_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn09297', 'rxn01236'],
  'num_missing': 2,
  'flux': 0.0,
  'blocked_reactions': ['rxn01236'],
  'dead_end_metabolites': ['cpd01662_c0']},
 {'start': 'cpd00149_e0',
  'path': ['cpd00149_e0', 

In [34]:
def build_reaction_graph(model):
    G = nx.DiGraph()
    for rxn in model.reactions:
        if rxn.upper_bound > 0:
            for reactant in rxn.reactants:
                for product in rxn.products:
                    G.add_edge(reactant.id, product.id, reaction=rxn.id)
        if rxn.lower_bound < 0:
            for product in rxn.products:
                for reactant in rxn.reactants:
                    G.add_edge(product.id, reactant.id, reaction=rxn.id)
    return G

def diagnose_gapfill_path_with_precursor_check(model, path_record, reaction_dict, 
                                                target_exchange_id="EX_cpd00211_e0",
                                                target_met_id="cpd00211_c0",
                                                verbose=True):
    """
    添加缺失反应后诊断为何丁酸 flux 为 0，并定位阻断前体代谢物
    """
    model_copy = copy.deepcopy(model)

    # Step 1: 添加缺失反应
    added = []
    for rxn_id in path_record.get("missing_reactions", []):
        if rxn_id not in model_copy.reactions:
            if rxn_id not in reaction_dict:
                raise ValueError(f"缺失反应 {rxn_id} 不在 reaction_dict 中")
            rxn = reaction_dict[rxn_id].copy()
            model_copy.add_reactions([rxn])
            added.append(rxn_id)

    # Step 2: 设置目标函数
    if target_exchange_id not in model_copy.reactions:
        raise ValueError(f"目标反应 {target_exchange_id} 不存在于模型中")

    ex_rxn = model_copy.reactions.get_by_id(target_exchange_id)
    ex_rxn.lower_bound = 0
    ex_rxn.upper_bound = 1000
    model_copy.objective = ex_rxn

    # Step 3: FBA 运行
    solution = model_copy.optimize()
    flux_value = solution.objective_value

    # Step 4: 分析 dead-end 和 blocked reactions
    dead_ends = find_dead_ends_manual(model_copy)
    path_dead_ends = [m for m in path_record.get("path", []) if m in dead_ends]

    blocked_info = {}
    rxns_to_check = path_record.get("missing_reactions", []) + path_record.get("blocked_reactions", [])
    if rxns_to_check:
        fva = flux_variability_analysis(model_copy, reaction_list=rxns_to_check, fraction_of_optimum=0.0)
        for rxn_id in rxns_to_check:
            if rxn_id in fva.index:
                min_flux, max_flux = fva.loc[rxn_id, ['minimum', 'maximum']]
                if min_flux == 0 and max_flux == 0:
                    rxn = model_copy.reactions.get_by_id(rxn_id)
                    upstream = [m.id for m in rxn.reactants]
                    downstream = [m.id for m in rxn.products]
                    blocked_info[rxn_id] = {
                        "reaction": rxn.reaction,
                        "upstream_dead": [m for m in upstream if m in dead_ends],
                        "downstream_dead": [m for m in downstream if m in dead_ends],
                        "bounds": (rxn.lower_bound, rxn.upper_bound)
                    }

    # Step 5: 如果 flux 为 0，找前体中断代谢物
    precursor_dead = []
    if flux_value == 0:
        G = build_reaction_graph(model_copy)
        if target_met_id in G:
            reachable_precursors = nx.ancestors(G, target_met_id)
            precursor_dead = [m for m in reachable_precursors if m in dead_ends]

    # 返回诊断结果
    diagnosis = {
        "added_reactions": added,
        "flux": flux_value,
        "dead_end_metabolites_in_path": path_dead_ends,
        "blocked_reactions": blocked_info,
        "precursor_dead_ends": precursor_dead,
    }

    if verbose:
        print(" 添加反应:", added)
        print(" 设置目标函数:", target_exchange_id)
        print(" 优化结果 flux =", flux_value)
        print(" 路径中 dead-end:", path_dead_ends)
        print(" 阻塞反应:")
        for rxn_id, info in blocked_info.items():
            print(f"  - {rxn_id}: {info['reaction']}")
            print(f"    upstream dead: {info['upstream_dead']}, downstream dead: {info['downstream_dead']}")
        if flux_value == 0:
            print(" 阻断的前体代谢物（无法支撑丁酸）:", precursor_dead)

    return diagnosis


In [36]:
for i, path in enumerate(ranked[:2]):
    print(f"\n 路径 {i+1}: start = {path['start']}")
    diagnose_gapfill_path_with_precursor_check(model, path, reaction_dict)


 路径 1: start = cpd00067_e0
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpbiwxwu_l.lp
Reading time = 0.02 seconds
: 688 rows, 1404 columns, 6586 nonzeros
 添加反应: ['rxn08183']
 设置目标函数: EX_cpd00211_e0
 优化结果 flux = 0.0
 路径中 dead-end: ['cpd00211_c0']
 阻塞反应:
 阻断的前体代谢物（无法支撑丁酸）: ['cpd02506_c0', 'cpd15320_c0', 'cpd19019_c0', 'cpd00802_c0', 'cpd15338_c0', 'cpd00068_c0', 'cpd00343_c0', 'cpd23234_c0', 'cpd00616_c0', 'cpd03606_c0', 'cpd11436_c0', 'cpd00014_c0', 'cpd15680_c0', 'cpd00790_c0', 'cpd15508_c0', 'cpd00773_c0', 'cpd00118_e0', 'cpd03560_c0', 'cpd00530_c0', 'cpd11593_c0', 'cpd02097_c0', 'cpd03559_c0', 'cpd00268_c0', 'cpd00371_c0', 'cpd00085_c0', 'cpd11586_c0', 'cpd00521_c0', 'cpd00810_c0', 'cpd11588_c0', 'cpd02820_c0', 'cpd15354_c0', 'cpd11459_c0', 'cpd00363_c0', 'cpd15329_c0', 'cpd01101_c0', 'cpd00079_c0', 'cpd00066_e0', 'cpd00434_c0', 'cpd15560_c0', 'cpd02338_c0', 'cpd00062_c0', 'cpd00541_c0', 'cpd03163_c0', 'cpd03831_c0', 'cpd10162_c0', 'cpd15315_c0', 

In [ ]:
def find_dead_end_metabolites(model):
    """
    判断 dead-end 代谢物：只有产物或只有底物。
    """
    dead_ends = []
    for met in model.metabolites:
        producing = 0
        consuming = 0
        for rxn in met.reactions:
            if met in rxn.reactants and rxn.lower_bound < 0:
                consuming += 1
            if met in rxn.products and rxn.upper_bound > 0:
                producing += 1
        if producing == 0 or consuming == 0:
            dead_ends.append(met.id)
    return dead_ends

def reaction_priority_score(rid, reaction_dict):
    score = 0
    if "EX_" in rid or "TR_" in rid:
        score -= 10
    rxn = reaction_dict[rid]
    score += len(rxn.reactants) + len(rxn.products)
    if abs(rxn.lower_bound) < 1e-6 and abs(rxn.upper_bound) < 1e-6:
        score += 5
    return score

from cobra.flux_analysis import find_blocked_reactions
import time


def recursive_gapfill_optimized(
    model, reaction_dict, path_record, max_depth=5, depth=0, added_rxns=None, time_limit=30
):
    if added_rxns is None:
        added_rxns = set()
    start_time = time.time()

    if depth > max_depth:
        return {
            **path_record,
            "rescued_reactions": list(added_rxns),
            "rescued_count": len(added_rxns),
            "rescued_flux": 0.0,
            "status": "max_depth_exceeded"
        }

    with model.copy() as m2:
        for rxn_id in path_record["missing_reactions"] + path_record["blocked_reactions"]:
            if rxn_id in reaction_dict and rxn_id not in m2.reactions:
                m2.add_reactions([reaction_dict[rxn_id].copy()])
                added_rxns.add(rxn_id)

        try:
            m2.objective = m2.reactions.get_by_id("EX_cpd00211_e0")
            sol = m2.optimize()
            flux = sol.objective_value if sol.status == "optimal" else 0.0
        except:
            flux = 0.0

        if flux > 1e-6:
            return {
                **path_record,
                "rescued_reactions": list(added_rxns),
                "rescued_count": len(added_rxns),
                "rescued_flux": flux,
                "status": "solved"
            }

        if time.time() - start_time > time_limit:
            return {
                **path_record,
                "rescued_reactions": list(added_rxns),
                "rescued_count": len(added_rxns),
                "rescued_flux": 0.0,
                "status": "timeout"
            }

        current_blocked = find_blocked_reactions(m2)
        current_deadends = find_dead_end_metabolites(m2)

        new_rxns = set()
        for rxn_id, rxn in reaction_dict.items():
            if rxn_id in added_rxns:
                continue
            for met in rxn.metabolites:
                if met.id in current_deadends:
                    new_rxns.add(rxn_id)
                    break
            if rxn_id in current_blocked:
                new_rxns.add(rxn_id)

        if not new_rxns:
            return {
                **path_record,
                "rescued_reactions": list(added_rxns),
                "rescued_count": len(added_rxns),
                "rescued_flux": 0.0,
                "status": "no_more_rxns"
            }

        sorted_rxns = sorted(
            new_rxns,
            key=lambda rid: reaction_priority_score(rid, reaction_dict)
        )

        best_result = None
        for rxn_id in sorted_rxns:
            result = recursive_gapfill_optimized(
                model=model,
                reaction_dict=reaction_dict,
                path_record=path_record,
                max_depth=max_depth,
                depth=depth + 1,
                added_rxns=added_rxns | {rxn_id},
                time_limit=time_limit
            )
            if result["rescued_flux"] > 1e-6:
                return result
            if not best_result or result["rescued_count"] > best_result["rescued_count"]:
                best_result = result

        return best_result


In [ ]:
rescued_results = []
for idx, item in enumerate([best_path]):
#for idx, item in enumerate(results):
    print(f"Processing {idx+1}/{len(results)}: {item['start']}")
    result = recursive_gapfill_optimized(
        model=model,
        reaction_dict=reaction_dict,
        path_record=item,
        max_depth=5
    )
    rescued_results.append(result)


Processing 1/42: cpd00067_e0
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmprrhfzj_d.lp
Reading time = 0.02 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmplo4gwsky.lp
Reading time = 0.00 seconds
: 689 rows, 1407 columns, 6597 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmphz8rpjem.lp
Reading time = 0.00 seconds
: 689 rows, 1407 columns, 6597 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpf8l

Process SpawnPoolWorker-9066:
Traceback (most recent call last):
  File "/Users/azddza/Library/Caches/pypoetry/virtualenvs/electromics-project-Ddk0_V8b-py3.10/lib/python3.10/site-packages/sympy/core/assumptions.py", line 499, in getit
    return self._assumptions[fact]
KeyError: 'zero'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/azddza/.pyenv/versions/3.10.13/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/azddza/.pyenv/versions/3.10.13/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/azddza/Library/Caches/pypoetry/virtualenvs/electromics-project-Ddk0_V8b-py3.10/lib/python3.10/site-packages/optlang/gurobi_interface.py", line 615, in __setstate__
    self.__init__(problem=problem)
  File "/Users/azddza/Library/Caches/pypoetry/vi

Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpmf280t63.lp
Reading time = 0.00 seconds
: 689 rows, 1407 columns, 6597 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpig3ldqht.lp
Reading time = 0.00 seconds
: 689 rows, 1407 columns, 6597 nonzeros


In [ ]:
rescued_results 

 最推荐的gapfill路径如下：
{'blocked_reactions': [],
 'dead_end_metabolites': ['cpd00211_e0', 'cpd00211_c0'],
 'flux': 0.0,
 'missing_reactions': ['rxn08183'],
 'num_missing': 1,
 'path': ['cpd00067_e0', 'cpd00211_c0'],
 'start': 'cpd00067_e0'}


是否所有 flux 都为 0: True


In [17]:
len(results)

42

## 对于缺失位点添加补救反应

In [13]:
rescued_results = []
original_results = results.copy()  

for idx, item in enumerate(original_results):
    print(f"Processing item {idx+1}/{len(original_results)}: {item['start']}")

    start = item['start']
    path = item['path']
    to_add_rxns = item['missing_reactions']
    flux = item['flux']
    blocked_in_path = item['blocked_reactions']
    dead_ends_in_path = item['dead_end_metabolites']

    record = {
        "start": start,
        "path": path,
        "missing_reactions": to_add_rxns,
        "num_missing": len(to_add_rxns),
        "flux": flux,
        "blocked_reactions": blocked_in_path,
        "dead_end_metabolites": dead_ends_in_path
    }

    # 尝试补救 dead-end
    extra_rxns = set()
    for rxn_id, rxn in reaction_dict.items():
        for met, coeff in rxn.metabolites.items():
            if met.id in dead_ends_in_path and coeff < 0:  # 消费 dead-end
                extra_rxns.add(rxn_id)
                break

    rescued_flux = 0.0
    if extra_rxns:
        with model.copy() as m2:
            # 添加缺失反应
            for rxn_id in to_add_rxns:
                if rxn_id in reaction_dict:
                    m2.add_reactions([reaction_dict[rxn_id].copy()])
            # 添加补救反应
            for rxn_id in extra_rxns:
                if rxn_id in reaction_dict and rxn_id not in m2.reactions:
                    m2.add_reactions([reaction_dict[rxn_id].copy()])

            try:
                m2.objective = m2.reactions.get_by_id("EX_cpd00211_e0")
                sol2 = m2.optimize()
                rescued_flux = sol2.objective_value if sol2.status == "optimal" else 0.0
            except:
                rescued_flux = 0.0

    # 添加补救信息
    record["rescued_reactions"] = list(extra_rxns)
    record["rescued_count"] = len(extra_rxns)
    record["rescued_flux"] = rescued_flux

    rescued_results.append(record)

# 可选：将补救信息合并进原始 results
# results.extend(rescued_results)


Processing item 1/42: cpd00001_e0
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpp9ddxgx0.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Processing item 2/42: cpd00048_e0
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp2gk8dskf.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Processing item 3/42: cpd00118_e0
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpsp8d20f7.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Processing item 4/42: cpd00092_e0
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmploa5y24y.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Processing item 5/42: cpd15603_e0
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp33ldtiw_.lp
Reading time = 0.00 seconds
: 688 rows, 1404 columns, 6586 nonzeros
Processing item

In [15]:
import pandas as pd
df = pd.DataFrame(rescued_results)
(df['rescued_flux'] == 1000).any()

np.False_

In [16]:
df

,start,path,missing_reactions,num_missing,flux,blocked_reactions,dead_end_metabolites,rescued_reactions,rescued_count,rescued_flux
0,cpd00001_e0,"[cpd00001_e0, cpd00001_c0, cpd00211_c0]","[rxn34181, rxn45696]",2,0.0,[rxn45696],"[cpd32214_c0, cpd24202_c0, cpd00211_c0]","[rxn43894, rxn11378, rxn01237, rxn23428, rxn00...",9,0.0
1,cpd00048_e0,"[cpd00048_e0, cpd00008_c0, cpd00211_c0]","[rxn29239, rxn01236]",2,0.0,[rxn01236],"[cpd01662_c0, cpd00048_e0]","[rxn09276, rxn05651, rxn34447, rxn05153, rxn29...",8,0.0
2,cpd00118_e0,"[cpd00118_e0, cpd00001_c0, cpd00211_c0]","[rxn42882, rxn45696]",2,0.0,[rxn45696],"[cpd32214_c0, cpd24202_c0, cpd00211_c0]","[rxn43894, rxn11378, rxn01237, rxn23428, rxn00...",9,0.0
3,cpd00092_e0,"[cpd00092_e0, cpd00067_c0, cpd00001_c0, cpd002...","[rxn09365, rxn48566, rxn45696]",3,0.0,"[rxn48566, rxn45696]","[cpd28082_c0, cpd35107_c0, cpd32214_c0, cpd000...","[rxn43467, rxn47781, rxn17210, rxn48165, rxn10...",339,0.0
4,cpd15603_e0,"[cpd15603_e0, cpd00008_c0, cpd00211_c0]","[rxn12848, rxn01236]",2,0.0,[rxn01236],"[cpd01662_c0, cpd15603_e0]",[rxn12848],1,0.0
5,cpd00156_e0,"[cpd00156_e0, cpd00008_c0, cpd00211_c0]","[rxn09373, rxn01236]",2,0.0,[rxn01236],"[cpd01662_c0, cpd00156_e0]","[rxn09373, rxn05242, rxn09374, rxn05245, rxn05...",7,0.0
6,cpd00166_e0,"[cpd00166_e0, cpd00008_c0, cpd00211_c0]","[rxn18671, rxn01236]",2,0.0,[rxn01236],"[cpd01662_c0, cpd00166_e0]","[rxn18671, rxn08070]",2,0.0
7,cpd03422_e0,"[cpd03422_e0, cpd00008_c0, cpd00211_c0]","[rxn10449, rxn01236]",2,0.0,[rxn01236],"[cpd01662_c0, cpd03422_c0, cpd03422_e0]","[rxn10090, rxn29150, rxn29151, rxn05029, rxn10...",7,0.0
8,cpd00104_e0,"[cpd00104_e0, cpd00008_c0, cpd00211_c0]","[rxn05223, rxn01236]",2,0.0,[rxn01236],"[cpd00104_e0, cpd01662_c0]","[rxn38792, rxn29122, rxn05223, rxn09693, rxn38...",5,0.0
9,cpd15240_e0,"[cpd15240_e0, cpd15240_c0, cpd00001_c0, cpd002...","[rxn09841, rxn36515, rxn45696]",3,0.0,"[rxn36515, rxn45696]","[cpd32214_c0, cpd24940_c0, cpd24202_c0, cpd002...","[rxn43894, rxn11378, rxn01237, rxn19112, rxn00...",12,0.0


In [18]:
# 显示成功补救通量的路径
df[df["rescued_flux"] > 0][["start", "rescued_flux", "rescued_count", "rescued_reactions"]]


,start,rescued_flux,rescued_count,rescued_reactions


In [92]:
# 假设你已经有 df，其中包含列 rescued_reactions
rescue_rxns = df[df["rescued_flux"] > 0]["rescued_reactions"].iloc[0]  # 提取第一行的反应列表
core_path_rxns = ["rxn31359", "rxn13427"]

minimal_set = set(rescue_rxns)
baseline_flux = 1000.0

for rxn in rescue_rxns:
    test_set = minimal_set - {rxn}

    with model.copy() as m:
        for r in core_path_rxns + list(test_set):
            if r in reaction_dict:
                m.add_reactions([reaction_dict[r].copy()])
        
        # 设置目标
        m.objective = m.reactions.get_by_id("EX_cpd00211_e0")
        m.reactions.get_by_id(f"EX_{start}").lower_bound = -10

        flux = m.optimize().objective_value
        if flux >= baseline_flux * 0.999:  # 允许一点浮动
            print(f" 移除 {rxn} 不影响 flux → 删除")
            minimal_set.remove(rxn)
        else:
            print(f" 移除 {rxn} 导致 flux 下降 → 保留")


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmphgcqle17.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn08184 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpscdr8p9_.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn35683 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp668ifcwi.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


...

 移除 rxn16586 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpsqg11cnx.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn43573 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmprnotss4s.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn43619 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpc2de4360.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn13427 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmptax9wfxb.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn31413 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpu5koggxw.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn03642 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpvqujjgjx.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn00994 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0_vlrtls.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn31412 不影响 flux → 删除
Read LP format model fr

In [93]:
minimal_set

{'rxn00875', 'rxn10074'}

In [94]:
final_rxns_to_add = core_path_rxns + list(minimal_set)
print(f" 最终要添加的反应总数: {len(final_rxns_to_add)}")


 最终要添加的反应总数: 4


In [96]:
target_rxn_id = "EX_cpd00211_e0"
with model.copy() as m:
    for rxn_id in final_rxns_to_add:
        if rxn_id in reaction_dict:
            m.add_reactions([reaction_dict[rxn_id].copy()])
    m.objective = target_rxn_id
    sol = m.optimize()
    flux = sol.objective_value if sol.status == "optimal" else 0
    print(f"\n 添加最终反应集后的通量: {flux}")


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp3we0w8rz.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros

 添加最终反应集后的通量: 1000.0


In [ ]:
from cobra.flux_analysis import flux_variability_analysis
# ✅ 主路径反应列表（根据你自己模型设定）
core_path_rxns = ["rxn31359", "rxn13427"]  # 示例，替换为你起点→目标路径的反应

# ✅ 补救反应集（可以从 DataFrame 中提取）
rescue_rxns = list(df[df["rescued_flux"] > 0]["rescued_reactions"].iloc[0])

with model.copy() as m:
    for r in core_path_rxns + rescue_rxns:
        if r in reaction_dict:
            m.add_reactions([reaction_dict[r].copy()])

    fva = flux_variability_analysis(m, reaction_list=rescue_rxns)

    essential_rxns = fva[(fva["maximum"].abs() > 1e-6) | (fva["minimum"].abs() > 1e-6)].index.tolist()
    print(f" 最小必需反应集大小: {len(essential_rxns)}")


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpesrfr49k.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpem3fk5hg.lp
Reading time = 0.00 seconds
: 978 rows, 2129 columns, 9563 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpcalsuv5a.lp
Reading time = 0.00 seconds
: 978 rows, 2129 columns, 9563 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpgfgfjrvm.lp
Reading time = 0.00 seconds
: 978 rows, 2129 columns, 9563 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - ex

In [84]:
enhanced_results = []

for res in results:
    start = res["start"]
    base_rxns = res["missing_reactions"]
    dead_mets = res["dead_end_metabolites"]

    for dead_met in dead_mets:
        try:
            # 反向图中从 dead_met → start 找最短路径
            reverse_path = nx.shortest_path(G.reverse(), source=dead_met, target=start)
            print(f"dead-end {dead_met} → 起点 {start} 路径: {reverse_path}")
        except:
            continue

        # 获取修复路径上的反应
        repair_rxns = []
        for i in range(len(reverse_path) - 1):
            try:
                rxn_id = G[reverse_path[i]][reverse_path[i + 1]]["reaction"]
                repair_rxns.append(rxn_id)
            except:
                continue

        # 合并所有需添加的反应
        all_rxns_to_add = list(set(base_rxns + repair_rxns))

        with model.copy() as m:
            for rxn_id in all_rxns_to_add:
                if rxn_id in reaction_dict:
                    new_rxn = reaction_dict[rxn_id].copy()
                    m.add_reactions([new_rxn])

            # 添加起点 exchange
            ex_id = f"EX_{start}"
            if ex_id not in m.reactions:
                try:
                    ex = Reaction(ex_id)
                    ex.lower_bound = -10
                    ex.upper_bound = 1000
                    ex.add_metabolites({m.metabolites.get_by_id(start): -1})
                    m.add_reactions([ex])
                except:
                    continue
            else:
                m.reactions.get_by_id(ex_id).lower_bound = -10

            # 添加目标产物 exchange
            target_ex_id = "EX_cpd00211_e0"
            if target_ex_id not in m.reactions:
                try:
                    target_met = m.metabolites.get_by_id("cpd00211_e0")
                    ex = Reaction(target_ex_id)
                    ex.lower_bound = 0
                    ex.upper_bound = 1000
                    ex.add_metabolites({target_met: -1})
                    m.add_reactions([ex])
                except:
                    continue
            m.objective = target_ex_id

            # 优化
            sol = m.optimize()
            flux = sol.objective_value if sol.status == "optimal" else 0
            print(f" 尝试修复 dead-end {dead_met} 后通量: {flux}")

            enhanced_results.append({
                "start": start,
                "original_dead_end": dead_met,
                "original_missing": base_rxns,
                "repair_rxns": repair_rxns,
                "flux_after_repair": flux
            })


dead-end cpd01662_c0 → 起点 cpd00254_e0 路径: ['cpd01662_c0', 'cpd00009_c0', 'cpd00254_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpm4sc39dk.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 尝试修复 dead-end cpd01662_c0 后通量: 0.0
dead-end cpd00205_e0 → 起点 cpd00205_e0 路径: ['cpd00205_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp07fyvdax.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 尝试修复 dead-end cpd00205_e0 后通量: 0.0
dead-end cpd01662_c0 → 起点 cpd00205_e0 路径: ['cpd01662_c0', 'cpd00009_c0', 'cpd00205_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0zhnosml.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 尝试修复 dead-end cpd01662_c0 后通量: 0.0
dead-end cpd00166_e0 → 起点 cpd00166_e0 路径: ['cpd00166_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0bbz5fg7.lp
Reading time = 0

In [87]:
import pandas as pd

dff = pd.DataFrame(enhanced_results)
dff


,start,original_dead_end,original_missing,repair_rxns,flux_after_repair
0,cpd00254_e0,cpd01662_c0,"[rxn10571, rxn01236]",[rxn00871],0.0
1,cpd00205_e0,cpd00205_e0,"[rxn18910, rxn01236]",[],0.0
2,cpd00205_e0,cpd01662_c0,"[rxn18910, rxn01236]","[rxn00871, rxn13842]",0.0
3,cpd00166_e0,cpd00166_e0,"[rxn18671, rxn01236]",[],0.0
4,cpd00166_e0,cpd01662_c0,"[rxn18671, rxn01236]",[rxn00871],0.0
5,cpd00322_e0,cpd00322_e0,"[rxn08745, rxn01236]",[],0.0
6,cpd00322_e0,cpd01662_c0,"[rxn08745, rxn01236]",[rxn00871],0.0
7,cpd00104_e0,cpd00104_e0,"[rxn05223, rxn01236]",[],0.0
8,cpd00104_e0,cpd01662_c0,"[rxn05223, rxn01236]","[rxn00871, rxn05223]",0.0
9,cpd10515_e0,cpd01662_c0,"[rxn08472, rxn01236]",[rxn00871],0.0
